# Evaluation of the Randomized Workload

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style("whitegrid")

## Data loading

Run 1:

In [2]:
df1 = pd.read_csv("workloads/randomized/job-complete-aqo-1-cout.csv")
df1.reset_index(inplace=True)
df1["index"] += 1
df1.sort_values(by="label", inplace=True)

Run 2:

In [3]:
df2 = pd.read_csv("workloads/randomized/job-complete-aqo-2-cout.csv")
df2.reset_index(inplace=True)
df2["index"] += 1
df2.sort_values(by="label", inplace=True)

Run 3:

In [4]:
df3 = pd.read_csv("workloads/randomized/job-complete-aqo-3-cout.csv")
df3.reset_index(inplace=True)
df3["index"] += 1
df3.sort_values(by="label", inplace=True)

## Detecting queries with different $C_{out}$ values

In [5]:
proj = ["cout", "t_exec", "label", "index"]
_m1 = pd.merge(df1[proj], df2[proj], on="label", suffixes=["_run1", "_run2"])
df_comb = pd.merge(_m1, df3[proj], on="label", suffixes=["", "_run3"])
df_comb.set_index("label", inplace=True)
df_comb.rename(columns={"cout": "cout_run3", "t_exec": "t_exec_run3", "index": "index_run3"}, inplace=True)

In [6]:
cout_cols = ["cout_run1", "index_run1", "cout_run2", "index_run2", "cout_run3", "index_run3"]
t_exec_cols = ["t_exec_run1", "index_run1", "t_exec_run2", "index_run2", "t_exec_run3", "index_run3"]

In [7]:
df_comb["diff_1_2"] = df_comb.cout_run1 != df_comb.cout_run2
df_comb["diff_1_3"] = df_comb.cout_run1 != df_comb.cout_run3
df_comb["diff_2_3"] = df_comb.cout_run2 != df_comb.cout_run3
df_comb["diff_any"] = df_comb.diff_1_2 | df_comb.diff_1_3 | df_comb.diff_2_3
df_comb[df_comb.diff_any][cout_cols]

,cout_run1,index_run1,cout_run2,index_run2,cout_run3,index_run3
label,,,,,,
13b,1912702,50,1912702,99,2641491,37
13c,2640445,26,2640445,72,1912521,108
16a,2602623,108,14366,26,14366,73
16d,216884,43,619573,50,619573,98
17b,399376,30,3599101,108,399376,70
17c,3533427,83,362078,55,3533427,113
22a,167972,101,492850,96,167972,32
22b,380673,25,163944,103,380673,26
2a,68686,41,804167,59,68546,107


In [8]:
df_comb[df_comb.diff_any][t_exec_cols]

,t_exec_run1,index_run1,t_exec_run2,index_run2,t_exec_run3,index_run3
label,,,,,,
13b,14931.466,50,15476.340,99,20618.351,37
13c,22531.254,26,20874.950,72,14728.999,108
16a,24687.911,108,14697.899,26,14672.818,73
16d,20714.552,43,20028.361,50,19205.697,98
17b,30549.297,30,27558.473,108,25974.502,70
17c,25371.218,83,23435.039,55,25159.056,113
22a,10055.220,101,11210.206,96,9831.744,32
22b,11267.370,25,10086.261,103,10638.097,26
2a,2575.391,41,5760.084,59,2319.825,107


---

# Evaluation of the Incremental Workload

In [1]:
import json

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style("whitegrid")

## Data loading

In [2]:
df_native = pd.read_csv("../ues/workloads/job-implicit-cout.csv")
df_run1 = pd.read_csv("workloads/incremental/job-full-run-1-cout.csv")
df_run2 = pd.read_csv("workloads/incremental/job-full-run-2-cout.csv")
df_run3 = pd.read_csv("workloads/incremental/job-full-run-3-cout.csv")
df_run1["run"] = 1
df_run1.plan = df_run1.plan.apply(json.loads)
df_run2["run"] = 2
df_run2.plan = df_run2.plan.apply(json.loads)
df_run3["run"] = 3
df_run3.plan = df_run3.plan.apply(json.loads)
df_aqo = pd.concat([df_run1, df_run2, df_run3])
df_aqo

,query,cout,plan,t_exec,t_plan,label,run
0,"SELECT COUNT(*) FROM char_name AS chn, cast_in...",38424,"[{'Plan': {'Node Type': 'Aggregate', 'Strategy...",23480.780,110.555,10a,1
1,"SELECT COUNT(*) FROM char_name AS chn, cast_in...",4,"[{'Plan': {'Node Type': 'Aggregate', 'Strategy...",10420.391,22.254,10b,1
2,"SELECT COUNT(*) FROM char_name AS chn, cast_in...",1260587,"[{'Plan': {'Node Type': 'Aggregate', 'Strategy...",19898.561,7.607,10c,1
3,"SELECT COUNT(*) FROM company_name AS cn, compa...",403299,"[{'Plan': {'Node Type': 'Aggregate', 'Strategy...",3835.188,72.951,11a,1
4,"SELECT COUNT(*) FROM company_name AS cn, compa...",1301,"[{'Plan': {'Node Type': 'Aggregate', 'Strategy...",19560.226,15.755,11b,1
...,...,...,...,...,...,...,...
108,"SELECT COUNT(*) FROM aka_name AS an1, cast_inf...",872726,"[{'Plan': {'Node Type': 'Aggregate', 'Strategy...",15853.688,6.219,8d,3
109,"SELECT COUNT(*) FROM aka_name AS an, char_name...",403264,"[{'Plan': {'Node Type': 'Aggregate', 'Strategy...",14105.930,18.249,9a,3
110,"SELECT COUNT(*) FROM aka_name AS an, char_name...",138596,"[{'Plan': {'Node Type': 'Aggregate', 'Strategy...",12357.023,15.859,9b,3
111,"SELECT COUNT(*) FROM aka_name AS an, char_name...",276606,"[{'Plan': {'Node Type': 'Aggregate', 'Strategy...",13456.577,11.716,9c,3


Query with maximum runtime during the first iteration:

In [3]:
df_aqo[df_aqo.label.isin(df_run1[df_run1.t_exec == df_run1.t_exec.max()].label)]

,query,cout,plan,t_exec,t_plan,label,run
63,"SELECT COUNT(*) FROM complete_cast AS cc, comp...",109305,"[{'Plan': {'Node Type': 'Aggregate', 'Strategy...",395029.166,797.454,27a,1
63,"SELECT COUNT(*) FROM complete_cast AS cc, comp...",227911,"[{'Plan': {'Node Type': 'Aggregate', 'Strategy...",8843.744,794.242,27a,2
63,"SELECT COUNT(*) FROM complete_cast AS cc, comp...",106677,"[{'Plan': {'Node Type': 'Aggregate', 'Strategy...",299218.889,788.212,27a,3


In [4]:
df_aqo.groupby("run").t_exec.describe()

,count,mean,std,min,25%,50%,75%,max
run,,,,,,,,
1,113.0,21072.411646,42058.050599,794.170,8971.738,14543.107,20091.482,395029.166
2,113.0,13573.078743,10158.025080,791.983,8451.576,11902.815,19747.033,76928.338
3,113.0,48604.716097,218978.744631,788.624,8557.865,11987.346,19129.957,1703686.905


## Comparison of the different workload iterations

In [5]:
grp_run = df_aqo.groupby("run")

Average query execution time (in seconds):

In [6]:
grp_run.t_exec.describe()["mean"] / 1000

run
1    21.072412
2    13.573079
3    48.604716
Name: mean, dtype: float64

Average query planning time (in seconds):

In [7]:
grp_run.t_plan.describe()["mean"] / 1000

run
1    0.192078
2    0.189988
3    0.189535
Name: mean, dtype: float64

Total workload execution time (**in minutes!**):

In [8]:
(grp_run.t_exec.sum() + grp_run.t_plan.sum()) / 60000

run
1    40.048122
2    25.920442
3    91.895839
dtype: float64

## Why do runtimes increase so much for the final run?

In [9]:
grp_query = df_aqo.groupby("label")

In [10]:
def t_exec_increase(grp):
    init_t_exec = grp[grp.run == 1].t_exec
    final_t_exec = grp[grp.run == 3].t_exec
    return final_t_exec / init_t_exec

First up, compare how much the runtimes actually increase between first and third run:

In [11]:
ser_rt_inc = grp_query.apply(t_exec_increase).reset_index(level=0).set_index("label").squeeze()
ser_rt_inc

label
10a    0.484103
10b    1.004212
10c    0.710340
11a    0.524268
11b    0.121757
         ...   
8d     0.808406
9a     1.074978
9b     1.025814
9c     0.905402
9d     0.973346
Name: t_exec, Length: 113, dtype: float64

In [12]:
ser_rt_inc.describe()

count    113.000000
mean       1.946048
std        6.187333
min        0.121757
25%        0.872765
50%        0.978664
75%        1.024933
max       58.033085
Name: t_exec, dtype: float64

In [13]:
df_aqo = df_aqo.join(ser_rt_inc, on="label", rsuffix="_increase")
df_aqo

,query,cout,plan,t_exec,t_plan,label,run,t_exec_increase
0,"SELECT COUNT(*) FROM char_name AS chn, cast_in...",38424,"[{'Plan': {'Node Type': 'Aggregate', 'Strategy...",23480.780,110.555,10a,1,0.484103
1,"SELECT COUNT(*) FROM char_name AS chn, cast_in...",4,"[{'Plan': {'Node Type': 'Aggregate', 'Strategy...",10420.391,22.254,10b,1,1.004212
2,"SELECT COUNT(*) FROM char_name AS chn, cast_in...",1260587,"[{'Plan': {'Node Type': 'Aggregate', 'Strategy...",19898.561,7.607,10c,1,0.710340
3,"SELECT COUNT(*) FROM company_name AS cn, compa...",403299,"[{'Plan': {'Node Type': 'Aggregate', 'Strategy...",3835.188,72.951,11a,1,0.524268
4,"SELECT COUNT(*) FROM company_name AS cn, compa...",1301,"[{'Plan': {'Node Type': 'Aggregate', 'Strategy...",19560.226,15.755,11b,1,0.121757
...,...,...,...,...,...,...,...,...
108,"SELECT COUNT(*) FROM aka_name AS an1, cast_inf...",872726,"[{'Plan': {'Node Type': 'Aggregate', 'Strategy...",15853.688,6.219,8d,3,0.808406
109,"SELECT COUNT(*) FROM aka_name AS an, char_name...",403264,"[{'Plan': {'Node Type': 'Aggregate', 'Strategy...",14105.930,18.249,9a,3,1.074978
110,"SELECT COUNT(*) FROM aka_name AS an, char_name...",138596,"[{'Plan': {'Node Type': 'Aggregate', 'Strategy...",12357.023,15.859,9b,3,1.025814
111,"SELECT COUNT(*) FROM aka_name AS an, char_name...",276606,"[{'Plan': {'Node Type': 'Aggregate', 'Strategy...",13456.577,11.716,9c,3,0.905402
